In [1]:
from ipyleaflet import Map, basemaps, CircleMarker
from ipywidgets import HTML
import pandas as pd
import sys
sys.path.append("water_security")
from data.labeled.preprocessed import  RISKS_MAPPING as risks
from data.labeled.preprocessed import SEVERITY_MAPPING
import numpy as np
from ipyleaflet import Icon, Marker, MarkerCluster, Popup

# First is latitude and second is longitude; both in degrees


m = Map(interpolation='nearest', basemap=basemaps.Stamen.Terrain,world_copy_jump=True)

markers = ()


popup = Popup(
            name='popup',
            close_button=False,
            auto_close=False,
            close_on_escape_key=False,auto_pan=False
        )
dpopup = Popup(
            name='download_popup',
            close_button=False,
            auto_close=False,
            close_on_escape_key=False,auto_pan=False
        )
def mouseover_callback(marker,location, html):
    def callback(*args, **kwargs):
        popup.child = html
        popup.location = location
        m.add_layer(popup)
    return callback

def mouseout_callback():
    def callback(*args, **kwargs):
        m.remove_layer(popup)
    return callback



def populate_per_city(city, country, risks_dict, pred_mask = None):
    html =  f"""
    <p> <h4><b> {city}, {country} </b></h4>
    
    """
    
    for risk in risks:
        if risk not in risks_dict:
            continue
        risk_value = risks_dict[risk]
        if not pd.isnull(risk_value) and risk_value !=0:
            risk_value = SEVERITY_MAPPING[np.round(risk_value).astype(int)]
            to_add = f"""
      <b>{risks[risk]}</b>:{risk_value}
   
    """
            if pred_mask is not None:
                if pred_mask[risk]:
                    to_add = f'<h4 style="color:blue">{to_add}</h4>'
                else:
                    to_add = f'<h4>{to_add}</h4>'
            html += to_add
            
    html += '</p>'
    return HTML(html)
    


    
from utils.geo import  is_close

In [2]:

from classification.model_handler import ModelHandler, InvalidCoordinates
from data.labeled.preprocessed import RISKS_MAPPING

handler = ModelHandler()

if not handler.is_fitted:
    handler.train()
    
risks_ids = sorted(RISKS_MAPPING)
from data.model.predictions import FILLED_DATASET as dataset ,PREDICTION_MASK as prediction_mask
for (_,city),(_,pred_mask) in zip(dataset.iterrows(),prediction_mask.iterrows()):
    location=(city['latitude'],city['longitude'])
    html = populate_per_city(city.city, city.country, city,pred_mask)
    marker = CircleMarker(location=location,fill_color = "blue",color='blue',radius=2)
    marker.on_mouseover(mouseover_callback(marker,location, html))
    marker.on_mouseout(mouseout_callback())
    markers = markers + (marker,)
    
from utils.geo import check_1k_population_density_source
import threading
threads = []
def on_download(coords):
    if len(threads) > 1:
        return True
    if len(threads) > 0:
        if threads[0].is_alive():
            m.remove_layer(dpopup)
            dpopup.child = HTML("Still Downloading..")
            dpopup.location = coords
            m.add_layer(dpopup)
            return True
        threads.pop()
    dthread = check_1k_population_density_source()
    if dthread is not None:
        threads.append(dthread)
        dpopup.child = HTML("Downloading required resource..")
        dpopup.location = coords
        m.add_layer(dpopup)
        return True
    return False

manually_added_markers = []

def handle_click(**kwargs):
    if kwargs.get('type') == 'click':
        coords = kwargs.get('coordinates')
        check = any(is_close((coords[0],coords[1]), x.location) for x in markers)
        if check:
            return
        if on_download(coords):
            return 
        try:
            output,mask = handler.test(coords[0], coords[1])
        except InvalidCoordinates:
            return
        html = populate_per_city("Close To:" + output['city'], output['country'],
                    {k: output[k] for k in risks_ids if k in output},
                    {k: mask[k] for k in risks_ids if k in mask})
        marker = CircleMarker(location=coords,fill_color = "red",color='red',radius=2)
        check = [is_close(marker.location,mark.location) for mark in manually_added_markers]
        if not any(check):
            
            marker.on_mouseover(mouseover_callback(marker,coords, html))
            marker.on_mouseout(mouseout_callback())
            manually_added_markers.append(marker)
            m.add_layer(marker)
        else:
            to_rem = [cnt for cnt,(m,c) in enumerate(zip(manually_added_markers,check)) if c][0]
            m.remove_layer(manually_added_markers[to_rem])
            manually_added_markers.pop(to_rem)
            m.remove_layer(popup)

m.on_interaction(handle_click)
m.add_layer(MarkerCluster(markers = markers))

## Instructions
### Click on clouds of cities to expand or hover over single points to view risks
- Risks shown in black are result of ground truth
- Risks shown in blue are result of prediction

### Click on the map for online prediction. Click the created red dot to remove.
(For online prediction, a file containing population densities is required to be downloaded)


In [3]:
m.layout.width = '80%'
m.layout.height = '1000px'
m.zoom=2.2
m.center = (0, 0)
display(m)

Map(center=[0, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text'…

['population']
They are going to be imputed.
['population']
They are going to be imputed.
['population']
They are going to be imputed.
['population']
They are going to be imputed.
['population']
They are going to be imputed.
['population']
They are going to be imputed.
['population']
They are going to be imputed.
['population']
They are going to be imputed.
['population']
They are going to be imputed.
['population']
They are going to be imputed.
['population']
They are going to be imputed.
['population']
They are going to be imputed.
['population']
They are going to be imputed.
['population']
They are going to be imputed.
